# Make counterbalanced sequences

In [1]:
import itertools
import numpy as np
import json 
import os
import random
import copy

In [2]:
def generate_latin_square(size):
    latin_square = [[0] * size for _ in range(size)]
    
    for row in range(size):
        for col in range(size):
            latin_square[row][col] = (row + col) % size + 1
    
    return latin_square

def print_latin_square(square):
    for row in square:
        print(row)

# Specify the size of the Latin square
size = 54  # Change this to the desired size

# Generate and print the Latin square
latin_square = generate_latin_square(size)
# print_latin_square(latin_square)

In [3]:
# blocks
block_v0 = [0.75, 1, 1.3, 1, 0.75, 1.3, 1.3, 1, 0.75] 
block_v1 = [1, 1.3, 0.75, 1.3, 1, 0.75, 0.75, 1.3, 1] 
block_v2 = [1.3, 0.75, 1, 0.75, 1.3, 1, 1, 0.75, 1.3] 

seq = block_v0 + block_v1 + block_v2
print(len(seq))

position = {0.75: [], 1: [], 1.3: []}
for i in range(len(seq)):
    position[seq[i]].append(i)
    
    
for key in position:
    print(str(key) + ': ' +  str(np.mean(np.array(position[key]))))

27
0.75: 13.0
1: 13.0
1.3: 13.0


In [4]:
# block orders:
# 1. 0 1 2 1 2 0
# 2. 2 0 1 0 1 2
# 3. 1 2 0 2 0 1

# the first three and last three are latin square counterbalanced across the three orders 
# i.e. ([0,1,2], [2, 0, 1], [1, 2, 0])

order_0 = block_v0 + block_v1 + block_v2 + block_v1 + block_v2 + block_v0
order_1 = block_v2 + block_v0 + block_v1 + block_v0 + block_v1 + block_v2
order_2 = block_v1 + block_v2 + block_v0 + block_v2 + block_v0 + block_v1

print(len(order_0))

orders = [order_0, order_1, order_2]

# each row of the latin square will be presented in each of these orders 


54


In [5]:
scenes = ['s' + str(i) for i in range(54)]
print(scenes)

['s0', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21', 's22', 's23', 's24', 's25', 's26', 's27', 's28', 's29', 's30', 's31', 's32', 's33', 's34', 's35', 's36', 's37', 's38', 's39', 's40', 's41', 's42', 's43', 's44', 's45', 's46', 's47', 's48', 's49', 's50', 's51', 's52', 's53']


In [6]:
# shuffle scenes so that images from the same unity environment are spread out 
random.shuffle(scenes)

scenes[0:8]

['s3', 's1', 's11', 's2', 's6', 's0', 's34', 's13']

In [7]:
# create the sequences with the scene name based off of the latin square 
sequences = {}
# row number
for row_num, row in enumerate(latin_square):
    for i in range(3):
        seq = []
        order = orders[i]
        for count, ele in enumerate(row):
            seq.append(scenes[ele-1] + '_' + str(order[count]) + 'x')
        seq_name = 'seq' + str(row_num) + '_' + str(i)
        sequences[seq_name] = seq

# 'seq0_0' - the first 0 refers to the sequence of which there are 54 (54 rows in the latin square), the second 0 refers to the order of which there are 3 (0, 1, 2)
# in total there are 54 sequences (0-53) and each one is presented at each order (0, 1, 2)
print(sequences['seq0_2'])

['s3_1x', 's1_1.3x', 's11_0.75x', 's2_1.3x', 's6_1x', 's0_0.75x', 's34_0.75x', 's13_1.3x', 's14_1x', 's47_1.3x', 's29_0.75x', 's28_1x', 's48_0.75x', 's19_1.3x', 's21_1x', 's8_1x', 's27_0.75x', 's15_1.3x', 's4_0.75x', 's35_1x', 's49_1.3x', 's7_1x', 's20_0.75x', 's31_1.3x', 's33_1.3x', 's52_1x', 's32_0.75x', 's18_1.3x', 's39_0.75x', 's51_1x', 's43_0.75x', 's24_1.3x', 's10_1x', 's36_1x', 's40_0.75x', 's41_1.3x', 's45_0.75x', 's23_1x', 's26_1.3x', 's12_1x', 's30_0.75x', 's16_1.3x', 's17_1.3x', 's5_1x', 's37_0.75x', 's46_1x', 's22_1.3x', 's42_0.75x', 's25_1.3x', 's50_1x', 's53_0.75x', 's38_0.75x', 's9_1.3x', 's44_1x']


In [8]:
# Need to get the full image name that includes the depth, and target pixel 
# imagename_ref contains the portion of the image name in the sequence (above) and the full image name
folder_path = '/Users/prachimahableshwarkar/Documents/GW/FacialAge/FacialAge_MTurk/BNav_EC2/DepthDuration/familiarSizeVE_MTurk/VR_target_stimuli'

file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

imagename_ref = {}
for file_name in file_names:
    if 'png' in file_name:
        imagename_ref[file_name.split('_')[0] + '_' + file_name.split('_')[1]] = file_name

imagename_ref['s13_1.3x']

's13_1.3x_2.4_313.png'

In [9]:
sequences_completeImageNames = {}

for seq in sequences:
    updated_seq = []
    for trial_image in sequences[seq]:
        updated_seq.append(imagename_ref[trial_image])
    sequences_completeImageNames[seq] = updated_seq
        

In [10]:
sequences['seq0_0'][:5]

['s3_0.75x', 's1_1x', 's11_1.3x', 's2_1x', 's6_0.75x']

In [11]:
sequences_completeImageNames['seq0_0'][:5]

['s3_0.75x_5_381.png',
 's1_1x_1.7_262.png',
 's11_1.3x_5_363.png',
 's2_1x_2.7_390.png',
 's6_0.75x_2.1_290.png']

In [12]:
len(sequences_completeImageNames), 54 * 3

(162, 162)

In [13]:
sequences_dicts_for_jsons = {}

durations = [125, 250, 1000]

for seq in sequences_completeImageNames:
    for dur in durations:
        complete_sequence = []
        for count, trial in enumerate(sequences_completeImageNames[seq]):
                trial_dict = {}
                trial_dict['sequence'] = seq + '_VE' + str(dur)
                trial_dict['image'] = trial
                trial_dict['duration'] = dur
                trial_dict['scene'] = trial.split('_')[0]
                trial_dict['num'] = count
#                 print(trial)
                trial_dict['depth'] = float(trial.split('_')[2])
                trial_dict['scale'] = trial.split('_')[1]
                # transform ypos based on the actual size of the image (height = 1680)
                height = 1680
                trial_dict['ypos'] = int(trial.split('_')[3][:-4])
                
                trial_dict['image_path_target'] = 'VR_target_stimuli/' + trial
                trial_dict['mask_path'] = 'masks/' + 'mask_' + str(count) + '.jpg'
                trial_dict['fixation_path'] = 'fixation.jpg'
                complete_sequence.append(trial_dict)
        sequences_dicts_for_jsons[seq + '_VE' + str(dur)] = complete_sequence


In [14]:
print(len(sequences_dicts_for_jsons), 54 * 3 * 3)
print(sequences_dicts_for_jsons['seq0_0_VE125'][0])
# print(sequences_dicts_for_jsons.keys())



486 486
{'sequence': 'seq0_0_VE125', 'image': 's3_0.75x_5_381.png', 'duration': 125, 'scene': 's3', 'num': 0, 'depth': 5.0, 'scale': '0.75x', 'ypos': 381, 'image_path_target': 'VR_target_stimuli/s3_0.75x_5_381.png', 'mask_path': 'masks/mask_0.jpg', 'fixation_path': 'fixation.jpg'}


In [15]:
final_sequences = copy.deepcopy(sequences_dicts_for_jsons)

## Add catch trials

In [16]:
# 54 total trials
# 6 catch trials
catch_trial_positions = [6, 16, 25, 33, 44, 52]
# diff = 10, 9, 8, 11, 8

In [17]:
all_catch_stim = ['c0.png', 'c1.png', 'c2.png', 'c3.png', 'c4.png', 'c5.png']
catch_trials_125 = []

for count, stim in enumerate(all_catch_stim):
    c_trial = {}
    c_trial['sequence'] = 'catch_trial'
    c_trial['image'] = stim
    
    c_trial['duration'] = 125
    
    c_trial['scene'] = 'NA'
    c_trial['num'] = count
    c_trial['depth'] = 'NA'
    c_trial['scale'] = 'NA'
    # have the image at center since there is no target
    ypos = 840//2
    c_trial['ypos'] = ypos
                
    c_trial['image_path_target'] = 'catch_stimuli/' + stim
    c_trial['mask_path'] = 'masks/' + 'mask_' + str(count) + '.jpg'
    c_trial['fixation_path'] = 'fixation.jpg'
    catch_trials_125.append(c_trial)
    
catch_trials_250 = []

for count, stim in enumerate(all_catch_stim):
    c_trial = {}
    c_trial['sequence'] = 'catch_trial'
    c_trial['image'] = stim
    
    c_trial['duration'] = 250
    
    c_trial['scene'] = 'NA'
    c_trial['num'] = count
    c_trial['depth'] = 'NA'
    c_trial['scale'] = 'NA'
    # have the image at center since there is no target
    ypos = 840//2
    c_trial['ypos'] = ypos
                
    c_trial['image_path_target'] = 'catch_stimuli/' + stim
    c_trial['mask_path'] = 'masks/' + 'mask_' + str(count) + '.jpg'
    c_trial['fixation_path'] = 'fixation.jpg'
    catch_trials_250.append(c_trial)
    
catch_trials_1000 = []

for count, stim in enumerate(all_catch_stim):
    c_trial = {}
    c_trial['sequence'] = 'catch_trial'
    c_trial['image'] = stim
    
    c_trial['duration'] = 1000
    
    c_trial['scene'] = 'NA'
    c_trial['num'] = count
    c_trial['depth'] = 'NA'
    c_trial['scale'] = 'NA'
    # have the image at center since there is no target
    ypos = 840//2
    c_trial['ypos'] = ypos
                
    c_trial['image_path_target'] = 'catch_stimuli/' + stim
    c_trial['mask_path'] = 'masks/' + 'mask_' + str(count) + '.jpg'
    c_trial['fixation_path'] = 'fixation.jpg'
    catch_trials_1000.append(c_trial)


In [18]:
len(catch_trials_125), catch_trials_125[0]

(6,
 {'sequence': 'catch_trial',
  'image': 'c0.png',
  'duration': 125,
  'scene': 'NA',
  'num': 0,
  'depth': 'NA',
  'scale': 'NA',
  'ypos': 420,
  'image_path_target': 'catch_stimuli/c0.png',
  'mask_path': 'masks/mask_0.jpg',
  'fixation_path': 'fixation.jpg'})

In [19]:
cdur_ref = {'125':catch_trials_125, '250':catch_trials_250, '1000':catch_trials_1000}

In [20]:
sequences_with_catchTrials = {}
for seqName in final_sequences:
    sequence = final_sequences[seqName]
    seq_duration = seqName.split('_')[-1][2:]
    catch_trials = cdur_ref[seq_duration]
    for i, position in enumerate(catch_trial_positions):
        # insert catch trial into the sequence at the position 
        sequence.insert(position, catch_trials[i])
    sequences_with_catchTrials[seqName] = sequence
    

In [21]:
# catch_trial_positions = [6, 16, 25, 33, 44, 52]

sequences_with_catchTrials['seq0_0_VE1000'][51:53]


[{'sequence': 'seq0_0_VE1000',
  'image': 's22_1x_4.29_354.png',
  'duration': 1000,
  'scene': 's22',
  'num': 46,
  'depth': 4.29,
  'scale': '1x',
  'ypos': 354,
  'image_path_target': 'VR_target_stimuli/s22_1x_4.29_354.png',
  'mask_path': 'masks/mask_46.jpg',
  'fixation_path': 'fixation.jpg'},
 {'sequence': 'catch_trial',
  'image': 'c5.png',
  'duration': 1000,
  'scene': 'NA',
  'num': 5,
  'depth': 'NA',
  'scale': 'NA',
  'ypos': 420,
  'image_path_target': 'catch_stimuli/c5.png',
  'mask_path': 'masks/mask_5.jpg',
  'fixation_path': 'fixation.jpg'}]

In [22]:
len(sequences_with_catchTrials['seq0_0_VE1000']), 54 + 6, len(sequences_with_catchTrials), 54 * 3 * 3

(60, 60, 486, 486)

## Save sequences as jsons

In [23]:
dest = '/Users/prachimahableshwarkar/Documents/GW/FacialAge/FacialAge_MTurk/BNav_EC2/DepthDuration/familiarSizeVE_MTurk/jsons/'
for seq in sequences_with_catchTrials:
    with open(dest + seq + ".json", "w") as outfile:
        json.dump(sequences_with_catchTrials[seq], outfile)

# Make practice sequence

In [68]:
file_names = ['p1_1x_4_304.png', 'p2_1.3x_3_371.png', 'p3_0.75x_1.4_278.png', 'c3.png']

practice_sequence = []
for count, trial in enumerate(file_names):
    if trial != 'c3.png':
        trial_dict = {}
        trial_dict['sequence'] = 'practice'
        trial_dict['image'] = trial
        trial_dict['duration'] = 250
        trial_dict['scene'] = trial.split('_')[0]
        trial_dict['num'] = count
        trial_dict['depth'] = float(trial.split('_')[2])
        trial_dict['scale'] = trial.split('_')[1]
        trial_dict['ypos'] = int(trial.split('_')[3][:-4])
        trial_dict['image_path_target'] = 'VR_PracticeStimuli/' + trial
        trial_dict['mask_path'] = 'masks/' + 'mask_' + str(count) + '.jpg'
        trial_dict['fixation_path'] = 'fixation.jpg'
        practice_sequence.append(trial_dict)
    else:
        trial_dict = {}
        trial_dict['sequence'] = 'practice'
        trial_dict['image'] = trial
        trial_dict['duration'] = 250
        trial_dict['scene'] = 'c3'
        trial_dict['num'] = count
        trial_dict['depth'] = 0
        trial_dict['scale'] = '0'
        trial_dict['ypos'] = 420
        trial_dict['image_path_target'] = 'VR_PracticeStimuli/' + trial
        trial_dict['mask_path'] = 'masks/' + 'mask_' + str(count) + '.jpg'
        trial_dict['fixation_path'] = 'fixation.jpg'
        practice_sequence.append(trial_dict)

In [69]:
practice_sequence

[{'sequence': 'practice',
  'image': 'p1_1x_4_304.png',
  'duration': 250,
  'scene': 'p1',
  'num': 0,
  'depth': 4.0,
  'scale': '1x',
  'ypos': 304,
  'image_path_target': 'VR_PracticeStimuli/p1_1x_4_304.png',
  'mask_path': 'masks/mask_0.jpg',
  'fixation_path': 'fixation.jpg'},
 {'sequence': 'practice',
  'image': 'p2_1.3x_3_371.png',
  'duration': 250,
  'scene': 'p2',
  'num': 1,
  'depth': 3.0,
  'scale': '1.3x',
  'ypos': 371,
  'image_path_target': 'VR_PracticeStimuli/p2_1.3x_3_371.png',
  'mask_path': 'masks/mask_1.jpg',
  'fixation_path': 'fixation.jpg'},
 {'sequence': 'practice',
  'image': 'p3_0.75x_1.4_278.png',
  'duration': 250,
  'scene': 'p3',
  'num': 2,
  'depth': 1.4,
  'scale': '0.75x',
  'ypos': 278,
  'image_path_target': 'VR_PracticeStimuli/p3_0.75x_1.4_278.png',
  'mask_path': 'masks/mask_2.jpg',
  'fixation_path': 'fixation.jpg'},
 {'sequence': 'practice',
  'image': 'c3.png',
  'duration': 250,
  'scene': 'c3',
  'num': 3,
  'depth': 0,
  'scale': '0',
  'y

In [60]:
# with open("practice_sequence.json", "w") as outfile:
#         json.dump(practice_sequence, outfile)

In [70]:
import json
with open("/Users/prachimahableshwarkar/Documents/GW/FacialAge/FacialAge_MTurk/BNav_EC2/DepthDuration/familiarSizeVE_MTurk/practice_sequence.json", "w") as outfile:
        json.dump(practice_sequence, outfile)

# Make Counterbalancing CSV

In [62]:
import os
import csv 

In [63]:
jsons_path = '/Users/prachimahableshwarkar/Documents/GW/FacialAge/FacialAge_MTurk/BNav_EC2/DepthDuration/familiarSizeVE_MTurk/jsons'


In [64]:
paths = []
for json in os.listdir(jsons_path):
    if '.json' in json:
        paths.append({'Path':'jsons/' + json, 'Sampled': 0})

In [65]:
# csv header
fieldnames = ['Path', 'Sampled']

# csv data
rows = paths

dest = '/Users/prachimahableshwarkar/Documents/GW/FacialAge/FacialAge_MTurk/BNav_EC2/DepthDuration/familiarSizeVE_MTurk/'

with open(dest + 'complete_counterbalancing.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

In [66]:
paths_250ms = []
for json in os.listdir(jsons_path):
    if 'VE250' in json:
        paths_250ms.append({'Path':'jsons/' + json, 'Sampled': 0})

In [67]:
# csv header
fieldnames = ['Path', 'Sampled']

# csv data
rows = paths_250ms

dest = '/Users/prachimahableshwarkar/Documents/GW/FacialAge/FacialAge_MTurk/BNav_EC2/DepthDuration/familiarSizeVE_MTurk/'

with open(dest + 'counterbalancing.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)